In [3]:
import pandas as pd
from utils.sheet_operator import expenses_sheet
from utils.gcp_operator import gcs_writer

In [5]:
df = expenses_sheet()

In [6]:
# ajustando a estrutura do DataFrame, removendo o cabeçalho e trazendo a 1a linha para as colunas
df.columns = df.iloc[0]
df = df[1:].reset_index().drop('index', axis=1)

In [7]:
# criando um dicionário para as categorias da planilha
categories_indices = df.index[df['filter'] == '1'].tolist()
categories_dict = dict()

for index in enumerate(categories_indices):
    next_index = df.index[-1] if index[0] == len(categories_indices)-1 else categories_indices[index[0]+1]
    category = df.iloc[index[1]]['Mês']
    categories_dict[category] = df.iloc[index[1]+1:next_index]['Mês'].tolist()

In [8]:
# removendo as categorias do DataFrame
df = df[df['filter'] != '1']
df = df.drop('filter', axis=1)

In [9]:
# ajustando a estrutura do DataFrame
df = df.transpose().reset_index()
df.columns = df.iloc[0]
df = df[1:].reset_index().drop('index', axis=1)

In [10]:
# ajustando as colunas de valores de gastos
for column in df.columns:
    if column != 'Mês':
        df[column] = df[column].str.strip().str.replace(r'^R\$', '', regex=True)
        df[column] = df[column].str.replace(r'\s+', '', regex=True)
        df[column] = df[column].str.replace(r'\.', '', regex=True)
        df[column] = df[column].str.replace(r',', '.', regex=True)
        df[column] = df[column].astype(float)

In [11]:
df['Mês'] = pd.to_datetime(df['Mês'])

/tmp/ipykernel_4176/3781474281.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Mês'] = pd.to_datetime(df['Mês'])


In [12]:
for i in categories_dict.keys():
    print(i)

DESPESAS COM PESSOAL
MENSAIS VARIÁVEIS - COBRADOS NO BOLETO CONDOMÍNIO
MENSAIS FIXO
MANUTENÇÃO E CONSERVAÇÃO - (Prováveis de ocorrer*)
DESPESAS DIVERSAS
DESPESAS ADMINISTRATIVAS


In [13]:
# criando dicionário para renomear colunas
rename_columns = {
    'Mês': 'mes',
    # DESPESAS COM PESSOAL
    'Total com Salários': 'total_salarios',
    'Total com Adiantamento salarial (sempre dia 20)': 'total_adiantamento_salarial',
    'Exame médico periódico': 'exame_medico_periodico',
    'Férias Vitamar': 'ferias_vitamar',
    'Férias Daniel': 'ferias_daniel',
    'Férias Fábio': 'ferias_fabio',
    'Férias Jailza': 'ferias_jailza',
    'Férias Joana': 'ferias_joana',
    'Férias Karla': 'ferias_karla',
    'Total com Décimo terceiro salário (1 parc - setembro)': 'decimo_terceiro_parcela_1',
    'Total com Décimo terceiro salário (2 parc - dezembro)': 'decimo_terceiro_parcela_2',
    'Ticket alimentação': 'ticket_alimentacao',
    'INSS': 'inss',
    'FGTS': 'fgts',
    'Pis sobre folha de pagamento': 'pis_folha_pagamento',
    'Vale Transporte': 'vale_transporte',
    'Seguro de vida dos funcionários': 'seguro_vida',
    'Uniformes': 'uniformes'
}

In [14]:
# renomeando colunas
columns = categories_dict['DESPESAS COM PESSOAL']
columns.insert(0, 'Mês')
df = df[columns].rename(columns=rename_columns)

In [15]:
gcs_writer(df, 'silver/expenses/employees.csv')

In [16]:
expenses_sheet()

,Unnamed: 0,CONDOMINIO DOS EDIFÍCOS CAIÇARA E CANDEIAS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,filter,Mês,01-2020,02-2020,03-2020,04-2020,05-2020,06-2020,07-2020,08-2020,09-2020,10-2020,11-2020,12-2020
1,1,DESPESAS COM PESSOAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Total com Salários,"R$ 7.000,00","R$ 0,12","R$ 0,82","R$ 0,15","R$ 0,65","R$ 0,80","R$ 0,37","R$ 0,16","R$ 0,98","R$ 0,67","R$ 0,32","R$ 0,55"
3,NaN,Total com Adiantamento salarial (sempre dia 20),"R$ 0,31","R$ 0,98","R$ 0,11","R$ 0,76","R$ 0,15","R$ 0,17","R$ 0,01","R$ 0,88","R$ 0,82","R$ 0,91","R$ 0,14","R$ 0,06"
4,NaN,Exame médico periódico,"R$ 0,84","R$ 0,87","R$ 0,65","R$ 0,22","R$ 0,22","R$ 0,62","R$ 0,71","R$ 0,13","R$ 0,80","R$ 0,03","R$ 0,84","R$ 0,67"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,NaN,Renovação alvará elevador,"R$ 0,50","R$ 0,54","R$ 0,78","R$ 0,34","R$ 0,75","R$ 0,72","R$ 0,11","R$ 0,67","R$ 0,55","R$ 0,33","R$ 0,26","R$ 0,13"
65,NaN,Honorários advocatícios,"R$ 0,63","R$ 0,43","R$ 0,65","R$ 0,45","R$ 0,35","R$ 0,34","R$ 0,70","R$ 0,26","R$ 0,90","R$ 0,44","R$ 0,81","R$ 0,88"
66,NaN,Cartório,"R$ 0,04","R$ 0,69","R$ 0,26","R$ 0,18","R$ 0,23","R$ 0,53","R$ 0,54","R$ 0,88","R$ 0,67","R$ 0,40","R$ 0,84","R$ 0,68"
67,NaN,Certificado Digital,"R$ 0,49","R$ 0,89","R$ 0,02","R$ 0,06","R$ 0,79","R$ 0,15","R$ 0,78","R$ 0,11","R$ 0,37","R$ 0,92","R$ 0,28","R$ 0,86"


In [17]:
from utils.gcp_operator import gcs_reader
gcs_reader('silver/expenses/employees.csv')

,mes,total_salarios,total_adiantamento_salarial,exame_medico_periodico,ferias_vitamar,ferias_daniel,ferias_fabio,ferias_jailza,ferias_joana,ferias_karla,decimo_terceiro_parcela_1,decimo_terceiro_parcela_2,ticket_alimentacao,inss,fgts,pis_folha_pagamento,vale_transporte,seguro_vida,uniformes
0,2020-01-01,7000.00,0.31,0.84,0.31,0.33,0.04,0.49,0.18,0.31,0.20,0.92,0.62,0.81,0.69,1.00,0.13,0.50,0.27
1,2020-02-01,0.12,0.98,0.87,0.14,0.35,0.54,0.87,0.64,0.91,0.82,0.24,0.89,0.99,0.58,0.21,0.38,0.68,0.83
2,2020-03-01,0.82,0.11,0.65,0.03,0.88,0.11,0.43,0.72,0.74,0.75,0.74,0.26,0.13,0.05,0.37,0.19,0.26,0.46
3,2020-04-01,0.15,0.76,0.22,0.58,0.34,0.69,0.93,0.50,0.06,0.11,0.11,0.41,0.10,0.94,0.18,0.88,0.67,0.60
4,2020-05-01,0.65,0.15,0.22,0.23,0.23,0.37,0.03,0.92,0.88,0.42,0.57,0.45,0.89,0.20,0.79,0.59,0.64,0.17
5,2020-06-01,0.80,0.17,0.62,0.59,0.66,0.10,0.35,0.80,0.15,0.29,0.82,0.47,0.85,0.86,0.79,0.12,0.48,0.22
6,2020-07-01,0.37,0.01,0.71,0.42,0.26,0.38,0.51,0.33,0.74,0.78,0.37,0.67,0.32,0.88,0.53,0.29,0.37,0.72
7,2020-08-01,0.16,0.88,0.13,0.87,0.16,0.71,0.92,0.25,0.08,0.82,0.06,0.76,0.77,0.80,0.75,0.07,0.53,0.36
8,2020-09-01,0.98,0.82,0.80,0.29,0.71,0.73,0.20,0.20,0.55,0.94,0.07,0.75,0.04,0.69,0.75,0.66,0.37,0.12
9,2020-10-01,0.67,0.91,0.03,0.75,0.57,0.81,0.69,0.70,0.24,0.86,0.63,0.94,0.04,0.16,0.26,0.59,0.54,0.05
